# Part 1:  Initialize Environment

In [1]:
# Importing libraries
from web3 import Web3
import ast
import random
from prettytable import PrettyTable
import json

### Connecting to Ganache

<p>Ganache is a personal blockchain for rapid Ethereum and Corda distributed application development.</p>

![](GanacheScreenShot.png)

In [2]:
# Connect to Ganache
from web3 import Web3
web3 = Web3(Web3.HTTPProvider('HTTP://127.0.0.1:7545'))
print("isConnected to Ganache:",web3.isConnected())


isConnected to Ganache: True


In [3]:
#gettin lists of accounts available, Account[0] will be like an auditor account
accounts_list = web3.eth.accounts
for i in accounts_list:
    print(i)

0x4Bd223C450df75Ed59534CEb00f73b36ac11F3E9
0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167
0xF2FcAC0f153DeAf82A838f02ec48Ee52061CcEee
0x31E4015E49121516dcFeB6F1693Ec7840163B7A4
0xcE5719aCb8936767A8eca5b97FCBE69D8A139976
0x761C18cA37daAEbc00e6E00E5822EBc576CE9FA3


In [4]:
#Receiver's Address will be first from the list
receiver_address = accounts_list[0]
print("Buyer's Address:",receiver_address)


Buyer's Address: 0x4Bd223C450df75Ed59534CEb00f73b36ac11F3E9


In [5]:
#Senders Address will be second from the list
sender_address = accounts_list[1]
print("Seller's Address:",sender_address)

Seller's Address: 0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167


# Part 2:  Initialize Blockchain Environment

In [6]:
with open('build/contracts/TradingAgreement.json') as f:
  migration = json.load(f)

migration_contract = migration['networks']['5777']['address']
migration_abi = migration['abi']

# print(migration_contract)
# print(migration_abi)

In [9]:
#Deploy Instance with: 
#truffle migrate 0xdf261C6f71D739b70Ab9877541b3C821AD027Ac6  10 110

In [10]:
contract_instance = web3.eth.contract(abi=migration_abi, address=migration_contract)

#### Some Basic Information about the company (call function(s))

In [11]:
#Information about sender and receiver

print(f'''
OwnerCompany : {contract_instance.functions.ownerReceiver().call()}
OwnerAddress : {contract_instance.functions.ownerCompanyName().call()}

SenderCompany: {contract_instance.functions.senderCompanyName().call()}
SenderAddress: {contract_instance.functions.senderAddress().call()}
''')


OwnerCompany : 0x4Bd223C450df75Ed59534CEb00f73b36ac11F3E9
OwnerAddress : Rob's Company

SenderCompany: Sally's Company
SenderAddress: 0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167



In [12]:
#Other constructor Information

print(f'''

T_Deployment    : {contract_instance.functions.T_Deployment().call()}
T_start         : {contract_instance.functions.T_start().call()}
T_end           : {contract_instance.functions.T_end().call()}

time_slot       : {contract_instance.functions.time_slot().call()}

orderPerSlot    : {contract_instance.functions.orderPerSlot().call()}
N_min           : {contract_instance.functions.N_min().call()} //orderPerSlot;
N_max           : {contract_instance.functions.N_max().call()} //orderPerSlot * ((T_end-T_start)/time_slot);

T_agreement     : {contract_instance.functions.T_agreement().call()}

timeProcessOrder: {contract_instance.functions.timeProcessOrder().call()}
''')



T_Deployment    : 0
T_start         : 10
T_end           : 110

time_slot       : 10

orderPerSlot    : 2
N_min           : 2 //orderPerSlot;
N_max           : 20 //orderPerSlot * ((T_end-T_start)/time_slot);

T_agreement     : 10

timeProcessOrder: 2



# Part 3: Making Orders

In [13]:
#Making 20 orders
#let all have same OrderTime = 11,12 for now and and let fix ETA = 16,17 lets say.
#with pair wise incrementation
time_start = contract_instance.functions.T_start().call()
time_end = contract_instance.functions.T_end().call()

orderTime = [(i,i+1) for i in range(time_start + 1,time_end,10)]
print("Slot wise OrderTime:",orderTime)

Slot wise OrderTime: [(11, 12), (21, 22), (31, 32), (41, 42), (51, 52), (61, 62), (71, 72), (81, 82), (91, 92), (101, 102)]


In [14]:
etaTime = [(j,j+1) for j in range(time_start + 6 ,time_end,10)]
print("Slot wise ETA:",etaTime)

Slot wise ETA: [(16, 17), (26, 27), (36, 37), (46, 47), (56, 57), (66, 67), (76, 77), (86, 87), (96, 97), (106, 107)]


In [15]:
#executing 20 orders
orderTime_Temp = []
for i in orderTime:
    for j in list(i):
        orderTime_Temp.append(j)

etaTime_Temp = []
for y in etaTime:
    for z in list(y):
        etaTime_Temp.append(z)

OrderEtalist= list(zip(orderTime_Temp,etaTime_Temp))
print(OrderEtalist)

[(11, 16), (12, 17), (21, 26), (22, 27), (31, 36), (32, 37), (41, 46), (42, 47), (51, 56), (52, 57), (61, 66), (62, 67), (71, 76), (72, 77), (81, 86), (82, 87), (91, 96), (92, 97), (101, 106), (102, 107)]


In [16]:
#Making Transaction (orders 2 * (time_end - timestart)/10) 
order_per_slot = contract_instance.functions.orderPerSlot().call()
total_slots = contract_instance.functions.total_Slots().call()
print('order_per_slot:',order_per_slot)
print('total_slots:',total_slots)
print()

order_schedule_list = []
for i in range(1,total_slots+1):
    for j in range(order_per_slot):
        order_schedule_list.append(i)


order_per_slot: 2
total_slots: 10



In [17]:

table = PrettyTable(['Serial No','TimeSlot', 'Time_of_order', 'ETA'])

print('Parameters: Time slot | Time_of_order | ETA')

for k,i in enumerate(list(zip(order_schedule_list,OrderEtalist))[:-2]):
    table.add_row([k+1,i[0],i[1][0],i[1][1]])
    contract_instance.functions.orderConsignment(i[0],i[1][0],i[1][1]).transact({'from':receiver_address})
print(table)

Parameters: Time slot | Time_of_order | ETA
+-----------+----------+---------------+-----+
| Serial No | TimeSlot | Time_of_order | ETA |
+-----------+----------+---------------+-----+
|     1     |    1     |       11      |  16 |
|     2     |    1     |       12      |  17 |
|     3     |    2     |       21      |  26 |
|     4     |    2     |       22      |  27 |
|     5     |    3     |       31      |  36 |
|     6     |    3     |       32      |  37 |
|     7     |    4     |       41      |  46 |
|     8     |    4     |       42      |  47 |
|     9     |    5     |       51      |  56 |
|     10    |    5     |       52      |  57 |
|     11    |    6     |       61      |  66 |
|     12    |    6     |       62      |  67 |
|     13    |    7     |       71      |  76 |
|     14    |    7     |       72      |  77 |
|     15    |    8     |       81      |  86 |
|     16    |    8     |       82      |  87 |
|     17    |    9     |       91      |  96 |
|     18    |   

In [18]:
# Making extra order
table = PrettyTable(['Serial No','TimeSlot', 'Time_of_order', 'ETA'])
table.add_row([k+1,i[0],i[1][0],i[1][1]])
print(table)
try:
    contract_instance.functions.orderConsignment(i[0],i[1][0],i[1][1]).transact({'from':receiver_address})
except ValueError as e:
    print(ast.literal_eval(str(e))['message'])

+-----------+----------+---------------+-----+
| Serial No | TimeSlot | Time_of_order | ETA |
+-----------+----------+---------------+-----+
|     18    |    9     |       92      |  97 |
+-----------+----------+---------------+-----+
VM Exception while processing transaction: revert Time Slot order Maxed


In [19]:
#checking flag continue
print(f'flagContinue :{contract_instance.functions.flagContinue().call()}')

flagContinue :True


In [20]:
table = PrettyTable(['Serial No','TimeSlot', 'Time_of_order', 'ETA'])

for j,i in enumerate(list(zip(order_schedule_list,OrderEtalist))[-2:]):
    table.add_row([k+j+2,i[0],i[1][0],i[1][1]])
    contract_instance.functions.orderConsignment(i[0],i[1][0],i[1][1]).transact({'from':receiver_address})

print(table)

+-----------+----------+---------------+-----+
| Serial No | TimeSlot | Time_of_order | ETA |
+-----------+----------+---------------+-----+
|     19    |    10    |      101      | 106 |
|     20    |    10    |      102      | 107 |
+-----------+----------+---------------+-----+


In [21]:
# Making extra order
table = PrettyTable(['Serial No','TimeSlot', 'Time_of_order', 'ETA'])
table.add_row([21,11,102,107])
print(table)
try:
    contract_instance.functions.orderConsignment(i[0],i[1][0],i[1][1]).transact({'from':receiver_address})
except ValueError as e:
    print(ast.literal_eval(str(e))['message'])

+-----------+----------+---------------+-----+
| Serial No | TimeSlot | Time_of_order | ETA |
+-----------+----------+---------------+-----+
|     21    |    11    |      102      | 107 |
+-----------+----------+---------------+-----+
VM Exception while processing transaction: revert Maximum Orders Reached!


In [22]:
#Checking Number of order executed. 
total_orders_executed = contract_instance.functions.OrderBookIncrementor().call()
print(f"Total Orders Executed: {total_orders_executed}")

Total Orders Executed: 20


In [23]:
#checking flag continue
print(f'flagContinue :{contract_instance.functions.flagContinue().call()}')

flagContinue :False


In [24]:
# Retriving random order numbers (2 actual and 1 with error i.e. with order number 0)

In [25]:
random_orders = [i+1 for i in range(total_orders_executed)]
_=random.sample(random_orders, 2)
_.append(total_orders_executed+1)

for i in _:
    print(f'''    
    
Random_Order_Number:{i}
consignmentnumber:{contract_instance.functions.orderBookConsignment(i).call()[0]}
timeSlotOfOrder  :{contract_instance.functions.orderBookConsignment(i).call()[1]}
timeOfOrder      :{contract_instance.functions.orderBookConsignment(i).call()[2]}
consignmentSender:{contract_instance.functions.orderBookConsignment(i).call()[3]}
ETA              :{contract_instance.functions.orderBookConsignment(i).call()[4]}
DeliveryReviewed :{contract_instance.functions.orderBookConsignment(i).call()[5]}
        ''')
    print()
    


    
    
Random_Order_Number:7
consignmentnumber:7
timeSlotOfOrder  :4
timeOfOrder      :41
consignmentSender:0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167
ETA              :46
DeliveryReviewed :False
        

    
    
Random_Order_Number:6
consignmentnumber:6
timeSlotOfOrder  :3
timeOfOrder      :32
consignmentSender:0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167
ETA              :37
DeliveryReviewed :False
        

    
    
Random_Order_Number:21
consignmentnumber:0
timeSlotOfOrder  :0
timeOfOrder      :0
consignmentSender:0x0000000000000000000000000000000000000000
ETA              :0
DeliveryReviewed :False
        



# Part 4: Receiving Orders

In [26]:
accepting_orders = {
        0: (True),
        1: (True),
        2: (True),
        3: (True),
        4: (True),
        5: (True),
        6: (True),
        7: (True),
        8: (True),
        9: (False)   
    }

In [27]:
#making random acceptance
acceptance_list = []
for i in range(total_orders_executed):
    acceptance_list.append(accepting_orders[random.randrange(0,10)])
    
print(acceptance_list)

[False, True, True, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, False, True]


In [28]:
#Receiving 20
#lets assume everything is in range ±ETA 
eta_list = []
time_to_receive_consignments = [(i,i+1) for i in range(time_start + 5,time_end,10)]
# print(time_to_receive_consignments)
for i in time_to_receive_consignments:
    for j in range(len(i)):
#         print(i[j])
        eta_list.append(i[j])
    
print(eta_list)

[15, 16, 25, 26, 35, 36, 45, 46, 55, 56, 65, 66, 75, 76, 85, 86, 95, 96, 105, 106]


In [29]:
# now executing the order
table = PrettyTable(['OrderNumber','Accepting', "Buyer's ETA","Seller's ETA"])

for i,j in enumerate(list(zip(acceptance_list,eta_list))[:-2]):
    if(j[0]==False):
#         print(i+1,j[0],0)
        table.add_row([i+1,j[0],0,j[1]])
        contract_instance.functions.senderETA(i+1,j[1]).transact({'from':sender_address})
        contract_instance.functions.receiverConsignement(i+1,j[0],0).transact({'from':receiver_address})
    else:
#         print(i+1,j[0],j[1])
        table.add_row([i+1,j[0],j[1],j[1]])
        contract_instance.functions.senderETA(i+1,j[1]).transact({'from':sender_address})
        contract_instance.functions.receiverConsignement(i+1,j[0],j[1]).transact({'from':receiver_address})
    
print(table)     

+-------------+-----------+-------------+--------------+
| OrderNumber | Accepting | Buyer's ETA | Seller's ETA |
+-------------+-----------+-------------+--------------+
|      1      |   False   |      0      |      15      |
|      2      |    True   |      16     |      16      |
|      3      |    True   |      25     |      25      |
|      4      |   False   |      0      |      26      |
|      5      |    True   |      35     |      35      |
|      6      |   False   |      0      |      36      |
|      7      |    True   |      45     |      45      |
|      8      |    True   |      46     |      46      |
|      9      |    True   |      55     |      55      |
|      10     |    True   |      56     |      56      |
|      11     |    True   |      65     |      65      |
|      12     |    True   |      66     |      66      |
|      13     |    True   |      75     |      75      |
|      14     |    True   |      76     |      76      |
|      15     |    True   |    

In [30]:
# Minor check to show that when the flagContinue is true, you cannot call the checkpeformace function.
print(f'ContractEndAgreement :{contract_instance.functions.ContractEndAgreement().call()}')

ContractEndAgreement :False


In [31]:
table = PrettyTable(['OrderNumber','Accepting', "Buyer's ETA","Seller's ETA"])
table.add_row([19,True,105,105])
table.add_row([20,True,106,106])
print(table)
contract_instance.functions.senderETA(19,105).transact({'from':sender_address})
contract_instance.functions.receiverConsignement(19,True,105).transact({'from':receiver_address})
contract_instance.functions.senderETA(20,106).transact({'from':sender_address})
contract_instance.functions.receiverConsignement(20,False,0).transact({'from':receiver_address})


+-------------+-----------+-------------+--------------+
| OrderNumber | Accepting | Buyer's ETA | Seller's ETA |
+-------------+-----------+-------------+--------------+
|      19     |    True   |     105     |     105      |
|      20     |    True   |     106     |     106      |
+-------------+-----------+-------------+--------------+


HexBytes('0x349776548d62618f483d824d71b7781736217a6b85a6d2bdcc3aa5b79800cb53')

In [32]:
# Minor check to show that when the flagContinue is true, you cannot call the checkpeformace function.
print(f'ContractEndAgreement :{contract_instance.functions.ContractEndAgreement().call()}')

ContractEndAgreement :True


In [33]:
#Now if we receive order, We should get error:
import ast
try:
    contract_instance.functions.receiverConsignement(20,True,106).transact({'from':receiver_address})
except ValueError as e:
    print(ast.literal_eval(str(e))['message'])

VM Exception while processing transaction: revert Decision is Made!


In [34]:
random_orders = [i+1 for i in range(total_orders_executed)]
_=random.sample(random_orders, 2)

print("Note: if the delivery was not accepted, the flagDecision will be final (True) i.e. not changes can be made ")
print("Delivery Status Notation: 0-None,1-Success,2-Failed,3-Disputed")

for i in _:
    print(f'''   
    
Random_Order_Number:{i}

consignmentnumber       : {contract_instance.functions.receivingConsignmentBook(i).call()[0]}
consignmentOrderTime    : {contract_instance.functions.receivingConsignmentBook(i).call()[1]}
consignmnetShippingTime : {contract_instance.functions.receivingConsignmentBook(i).call()[2]}
ETAasPerOrder           : {contract_instance.functions.receivingConsignmentBook(i).call()[3]}
RobETA                  : {contract_instance.functions.receivingConsignmentBook(i).call()[4]}
SallyETA                : {contract_instance.functions.receivingConsignmentBook(i).call()[5]}
consignmentSender       : {contract_instance.functions.receivingConsignmentBook(i).call()[6]}
DeliveryStatusOnDelivery: {contract_instance.functions.receivingConsignmentBook(i).call()[7]}
flagDecision            : {contract_instance.functions.receivingConsignmentBook(i).call()[8]}
''')

    

Note: if the delivery was not accepted, the flagDecision will be final (True) i.e. not changes can be made 
Delivery Status Notation: 0-None,1-Success,2-Failed,3-Disputed
   
    
Random_Order_Number:4

consignmentnumber       : 4
consignmentOrderTime    : 0
consignmnetShippingTime : 0
ETAasPerOrder           : 27
RobETA                  : 0
SallyETA                : 26
consignmentSender       : 0x0000000000000000000000000000000000000000
DeliveryStatusOnDelivery: 2
flagDecision            : True

   
    
Random_Order_Number:1

consignmentnumber       : 1
consignmentOrderTime    : 0
consignmnetShippingTime : 0
ETAasPerOrder           : 16
RobETA                  : 0
SallyETA                : 15
consignmentSender       : 0x0000000000000000000000000000000000000000
DeliveryStatusOnDelivery: 2
flagDecision            : True



In [35]:
# Finalizing the Deal (verifying with ETA)
total_orders_received = contract_instance.functions.N().call() #both accepted and rejected

for i in range(len(acceptance_list)):
    try:
        contract_instance.functions.settingFlagDecision(i+1).transact({'from':receiver_address})
    except ValueError as e:
        print(ast.literal_eval(str(e))['message'])
    
    
       
    
    


VM Exception while processing transaction: revert Decision is Made!
VM Exception while processing transaction: revert Decision is Made!
VM Exception while processing transaction: revert Decision is Made!
VM Exception while processing transaction: revert Decision is Made!


In [36]:
print(k)

17


In [37]:
random_orders = [i+1 for i in range(total_orders_executed)]
_=random.sample(random_orders, 2)

print("Note: if the delivery was not accepted, the flagDecision will be final (True) i.e. not changes can be made ")
print("Delivery Status Notation: 0-None,1-Success,2-Failed,3-Disputed")

for i in _:
    print(f'''   
    
Random_Order_Number:{i}

consignmentnumber       : {contract_instance.functions.receivingConsignmentBook(i).call()[0]}
consignmentOrderTime    : {contract_instance.functions.receivingConsignmentBook(i).call()[1]}
consignmnetShippingTime : {contract_instance.functions.receivingConsignmentBook(i).call()[2]}
ETAasPerOrder           : {contract_instance.functions.receivingConsignmentBook(i).call()[3]}
RobETA                  : {contract_instance.functions.receivingConsignmentBook(i).call()[4]}
SallyETA                : {contract_instance.functions.receivingConsignmentBook(i).call()[5]}
consignmentSender       : {contract_instance.functions.receivingConsignmentBook(i).call()[6]}
DeliveryStatusOnDelivery: {contract_instance.functions.receivingConsignmentBook(i).call()[7]}
flagDecision            : {contract_instance.functions.receivingConsignmentBook(i).call()[8]}
''')


Note: if the delivery was not accepted, the flagDecision will be final (True) i.e. not changes can be made 
Delivery Status Notation: 0-None,1-Success,2-Failed,3-Disputed
   
    
Random_Order_Number:20

consignmentnumber       : 20
consignmentOrderTime    : 0
consignmnetShippingTime : 0
ETAasPerOrder           : 107
RobETA                  : 0
SallyETA                : 106
consignmentSender       : 0x0000000000000000000000000000000000000000
DeliveryStatusOnDelivery: 2
flagDecision            : True

   
    
Random_Order_Number:19

consignmentnumber       : 19
consignmentOrderTime    : 0
consignmnetShippingTime : 0
ETAasPerOrder           : 106
RobETA                  : 105
SallyETA                : 105
consignmentSender       : 0x0000000000000000000000000000000000000000
DeliveryStatusOnDelivery: 1
flagDecision            : True



In [38]:
random_orders = [i+1 for i in range(total_orders_executed)]
_=random.sample(random_orders, 2)

print("ORDERBOOK:")
for i in _:
    print(f'''    

Random_Order_Number:{i}
consignmentnumber:{contract_instance.functions.orderBookConsignment(i).call()[0]}
timeSlotOfOrder  :{contract_instance.functions.orderBookConsignment(i).call()[1]}
timeOfOrder      :{contract_instance.functions.orderBookConsignment(i).call()[2]}
consignmentSender:{contract_instance.functions.orderBookConsignment(i).call()[3]}
ETA              :{contract_instance.functions.orderBookConsignment(i).call()[4]}
DeliveryReviewed :{contract_instance.functions.orderBookConsignment(i).call()[5]}
        ''')
    print()
    

ORDERBOOK:
    

Random_Order_Number:11
consignmentnumber:11
timeSlotOfOrder  :6
timeOfOrder      :61
consignmentSender:0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167
ETA              :66
DeliveryReviewed :True
        

    

Random_Order_Number:8
consignmentnumber:8
timeSlotOfOrder  :4
timeOfOrder      :42
consignmentSender:0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167
ETA              :47
DeliveryReviewed :True
        



#### Reteriving Information from the blockchain

In [39]:
print(f'''
N                    : {contract_instance.functions.N().call()}
s                    : {contract_instance.functions.s().call()}
f                    : {contract_instance.functions.f().call()}
d                    : {contract_instance.functions.d().call()}
flagContinue         : {contract_instance.functions.flagContinue().call()}
''')


N                    : 20
s                    : 16
f                    : 4
d                    : 0
flagContinue         : False



In [40]:
#verification
assert(contract_instance.functions.N().call() == contract_instance.functions.s().call() + contract_instance.functions.f().call() + contract_instance.functions.d().call())
print(True)

True


In [41]:
random_orders = [i+1 for i in range(total_orders_executed)]
_=random.sample(random_orders, 2)
_.append(total_orders_executed+1)


for i in _:
    print(f'''
Random_Order_Number:{i}
consignmentnumber:{contract_instance.functions.orderBookConsignment(i).call()[0]}
timeSlotOfOrder  :{contract_instance.functions.orderBookConsignment(i).call()[1]}
timeOfOrder      :{contract_instance.functions.orderBookConsignment(i).call()[2]}
consignmentSender:{contract_instance.functions.orderBookConsignment(i).call()[3]}
ETA              :{contract_instance.functions.orderBookConsignment(i).call()[4]}
DeliveryReviewed :{contract_instance.functions.orderBookConsignment(i).call()[5]}
        ''')
    print()
    



Random_Order_Number:17
consignmentnumber:17
timeSlotOfOrder  :9
timeOfOrder      :91
consignmentSender:0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167
ETA              :96
DeliveryReviewed :True
        


Random_Order_Number:19
consignmentnumber:19
timeSlotOfOrder  :10
timeOfOrder      :101
consignmentSender:0x5A282A8025e6aAC79Ac6a8e01b462C3aC2849167
ETA              :106
DeliveryReviewed :True
        


Random_Order_Number:21
consignmentnumber:0
timeSlotOfOrder  :0
timeOfOrder      :0
consignmentSender:0x0000000000000000000000000000000000000000
ETA              :0
DeliveryReviewed :False
        



# Part 5: Scenario with a dispute

<p>We "Redeployed" the contract with the same set of parameters and made orders. (With new GANACHE environment)</p>

In [75]:

# Connect to Ganache
from web3 import Web3
web3 = Web3(Web3.HTTPProvider('HTTP://127.0.0.1:7545'))
print("isConnected to Ganache:",web3.isConnected())


isConnected to Ganache: True


In [76]:
#gettin lists of accounts available, Account[0] will be like an auditor account
accounts_list = web3.eth.accounts
for i in accounts_list:
    print(i)

0x2F8B45eC52EA714B3edB8123897d146871D4B8Ba
0x1B6296f8fe970277e91fB66F54B0031207011766
0x32a59DE6450485D8ecF36284c173f82546008e66
0x2ff6f2D7d65cdE81AA312F8a0f4600dE74cAc8bA
0xea28b4Ae8d2dA8F9e892c969F51db173210B49A5
0xcD726FC38bA747112F476429EC1840c8a33F3fba


In [77]:
#Receiver's Address will be first from the list
receiver_address = accounts_list[0]
print("Buyer's Address:",receiver_address)


Buyer's Address: 0x2F8B45eC52EA714B3edB8123897d146871D4B8Ba


In [78]:
#Senders Address will be second from the list
sender_address = accounts_list[1]
print("Seller's Address:",sender_address)

Seller's Address: 0x1B6296f8fe970277e91fB66F54B0031207011766


In [79]:
#executing 20 orders
orderTime_Temp = []
for i in orderTime:
    for j in list(i):
        orderTime_Temp.append(j)

etaTime_Temp = []
for y in etaTime:
    for z in list(y):
        etaTime_Temp.append(z)

OrderEtalist= list(zip(orderTime_Temp,etaTime_Temp))
print(OrderEtalist)

[(11, 16), (12, 17), (21, 26), (22, 27), (31, 36), (32, 37), (41, 46), (42, 47), (51, 56), (52, 57), (61, 66), (62, 67), (71, 76), (72, 77), (81, 86), (82, 87), (91, 96), (92, 97), (101, 106), (102, 107)]


In [85]:
with open('build/contracts/TradingAgreement.json') as f:
  migration = json.load(f)

migration_contract = migration['networks']['5777']['address']
migration_abi = migration['abi']

contract_instance = web3.eth.contract(abi=migration_abi, address=migration_contract)

#### Some Basic Information about the company (call function(s))

In [87]:
#Information about sender and receiver

print(f'''
OwnerCompany : {contract_instance.functions.ownerReceiver().call()}
OwnerAddress : {contract_instance.functions.ownerCompanyName().call()}

SenderCompany: {contract_instance.functions.senderCompanyName().call()}
SenderAddress: {contract_instance.functions.senderAddress().call()}
''')


OwnerCompany : 0x2F8B45eC52EA714B3edB8123897d146871D4B8Ba
OwnerAddress : Rob's Company

SenderCompany: Sally's Company
SenderAddress: 0x1B6296f8fe970277e91fB66F54B0031207011766



In [88]:
#Other constructor Information

print(f'''

T_Deployment    : {contract_instance.functions.T_Deployment().call()}
T_start         : {contract_instance.functions.T_start().call()}
T_end           : {contract_instance.functions.T_end().call()}

time_slot       : {contract_instance.functions.time_slot().call()}

orderPerSlot    : {contract_instance.functions.orderPerSlot().call()}
N_min           : {contract_instance.functions.N_min().call()} //orderPerSlot;
N_max           : {contract_instance.functions.N_max().call()} //orderPerSlot * ((T_end-T_start)/time_slot);

T_agreement     : {contract_instance.functions.T_agreement().call()}

timeProcessOrder: {contract_instance.functions.timeProcessOrder().call()}
''')



T_Deployment    : 0
T_start         : 10
T_end           : 110

time_slot       : 10

orderPerSlot    : 2
N_min           : 2 //orderPerSlot;
N_max           : 20 //orderPerSlot * ((T_end-T_start)/time_slot);

T_agreement     : 10

timeProcessOrder: 2



# Part 5.1: Making orders

In [89]:
#Making 20 orders
#let all have same OrderTime = 11,12 for now and and let fix ETA = 16,17 lets say.
#with pair wise incrementation

orderTime = [(i,i+1) for i in range(time_start + 1,time_end,10)]
print("Slot wise OrderTime:",orderTime)

Slot wise OrderTime: [(11, 12), (21, 22), (31, 32), (41, 42), (51, 52), (61, 62), (71, 72), (81, 82), (91, 92), (101, 102)]


In [90]:
etaTime = [(j,j+1) for j in range(time_start + 6 ,time_end,10)]
print("Slot wise ETA:",etaTime)

Slot wise ETA: [(16, 17), (26, 27), (36, 37), (46, 47), (56, 57), (66, 67), (76, 77), (86, 87), (96, 97), (106, 107)]


In [91]:
#Making Transaction (orders 2 * (time_end - timestart)/10) 
order_per_slot = contract_instance.functions.orderPerSlot().call()
total_slots = contract_instance.functions.total_Slots().call()
print('order_per_slot:',order_per_slot)
print('total_slots:',total_slots)
print()

order_schedule_list = []
for i in range(1,total_slots+1):
    for j in range(order_per_slot):
        order_schedule_list.append(i)


order_per_slot: 2
total_slots: 10



In [92]:

table = PrettyTable(['Serial No','TimeSlot', 'Time_of_order', 'ETA'])

print('Parameters: Time slot | Time_of_order | ETA')

for k,i in enumerate(list(zip(order_schedule_list,OrderEtalist))):
    table.add_row([k+1,i[0],i[1][0],i[1][1]])
    contract_instance.functions.orderConsignment(i[0],i[1][0],i[1][1]).transact({'from':receiver_address})
print(table)

Parameters: Time slot | Time_of_order | ETA
+-----------+----------+---------------+-----+
| Serial No | TimeSlot | Time_of_order | ETA |
+-----------+----------+---------------+-----+
|     1     |    1     |       11      |  16 |
|     2     |    1     |       12      |  17 |
|     3     |    2     |       21      |  26 |
|     4     |    2     |       22      |  27 |
|     5     |    3     |       31      |  36 |
|     6     |    3     |       32      |  37 |
|     7     |    4     |       41      |  46 |
|     8     |    4     |       42      |  47 |
|     9     |    5     |       51      |  56 |
|     10    |    5     |       52      |  57 |
|     11    |    6     |       61      |  66 |
|     12    |    6     |       62      |  67 |
|     13    |    7     |       71      |  76 |
|     14    |    7     |       72      |  77 |
|     15    |    8     |       81      |  86 |
|     16    |    8     |       82      |  87 |
|     17    |    9     |       91      |  96 |
|     18    |   

In [93]:
#Checking Number of order executed. 
total_orders_executed = contract_instance.functions.OrderBookIncrementor().call()
print(f"Total Orders Executed: {total_orders_executed}")
#checking flag continue
print(f'flagContinue :{contract_instance.functions.flagContinue().call()}')

Total Orders Executed: 20
flagContinue :False


# Part 5.2: Receiving Orders
<p>We will receive 19 orders at random and one order we will specify as a disputed order. </p>
<p>We plan to have 19th order as a disputed</p>
<p>Scenario:<br> ETA(as per order) 106
Seller will claim the ETA at 106 but Buyer doesn't agree and sets ETA at 109 (which is out of range of margined ETA)
<p>

In [94]:
# now executing the order
table = PrettyTable(['OrderNumber','Accepting', "Buyer's ETA","Seller's ETA"])

for i,j in enumerate(list(zip(acceptance_list,eta_list))[:-2]):
    if(j[0]==False):
#         print(i+1,j[0],0)
        table.add_row([i+1,j[0],0,j[1]])
        contract_instance.functions.senderETA(i+1,j[1]).transact({'from':sender_address})
        contract_instance.functions.receiverConsignement(i+1,j[0],0).transact({'from':receiver_address})
    else:
#         print(i+1,j[0],j[1])
        table.add_row([i+1,j[0],j[1],j[1]])
        contract_instance.functions.senderETA(i+1,j[1]).transact({'from':sender_address})
        contract_instance.functions.receiverConsignement(i+1,j[0],j[1]).transact({'from':receiver_address})

table.add_row([19,True,109,106])
table.add_row([20,False,0,107])
contract_instance.functions.senderETA(19,106).transact({'from':sender_address})
contract_instance.functions.receiverConsignement(19,True,109).transact({'from':receiver_address})
contract_instance.functions.senderETA(20,106).transact({'from':sender_address})
contract_instance.functions.receiverConsignement(20,False,0).transact({'from':receiver_address})

print(table)  

+-------------+-----------+-------------+--------------+
| OrderNumber | Accepting | Buyer's ETA | Seller's ETA |
+-------------+-----------+-------------+--------------+
|      1      |   False   |      0      |      15      |
|      2      |    True   |      16     |      16      |
|      3      |    True   |      25     |      25      |
|      4      |   False   |      0      |      26      |
|      5      |    True   |      35     |      35      |
|      6      |   False   |      0      |      36      |
|      7      |    True   |      45     |      45      |
|      8      |    True   |      46     |      46      |
|      9      |    True   |      55     |      55      |
|      10     |    True   |      56     |      56      |
|      11     |    True   |      65     |      65      |
|      12     |    True   |      66     |      66      |
|      13     |    True   |      75     |      75      |
|      14     |    True   |      76     |      76      |
|      15     |    True   |    

In [95]:
# Finalizing the Deal (of orderbook)
total_orders_received = contract_instance.functions.N().call() #both accepted and rejected

for i in range(len(acceptance_list)):
    try:
        contract_instance.functions.settingFlagDecision(i+1).transact({'from':receiver_address})
    except ValueError as e:
        print(ast.literal_eval(str(e))['message'])

VM Exception while processing transaction: revert Decision is Made!
VM Exception while processing transaction: revert Decision is Made!
VM Exception while processing transaction: revert Decision is Made!
VM Exception while processing transaction: revert Decision is Made!


In [96]:


print(f'''
N                    : {contract_instance.functions.N().call()}
s                    : {contract_instance.functions.s().call()}
f                    : {contract_instance.functions.f().call()}
d                    : {contract_instance.functions.d().call()}
flagContinue         : {contract_instance.functions.flagContinue().call()}
''')


N                    : 20
s                    : 15
f                    : 4
d                    : 1
flagContinue         : False



# Part 5.3: Resolving the dispute

<p>In order to resolve dispute between Sender and Buyer for ETA, there is an off-chain agreement between the two and they agree on some ETA, both entities'signature is added on the blockchain and the issue is resolved. <p>
    
<p>In order to resolve the dispute, both's Signature is necessary, else it will throw error.</p>

In [97]:
disputed_Consignment_number = 19

# Taking Seller's Signature  
_ = contract_instance.functions.sallySignature(disputed_Consignment_number).transact({'from':sender_address})
print("Transaction Hash:",_.hex())
# Taking Buyer's Signature
_ = contract_instance.functions.RobSignature(disputed_Consignment_number).transact({'from':receiver_address})
print("Transaction Hash:",_.hex())



Transaction Hash: 0xb85a1aead31a9ee39a95b1cc12672100073ac5503ef68b33a6f26cc5bb38a410
Transaction Hash: 0x701d6ed5bfe2683827ddfa85f3b36dc4a601d06525b438a275b5dfa8c7880fd8


In [98]:
# Resolving Dispute as "True"
_ = contract_instance.functions.resolveDispute(disputed_Consignment_number,True).transact({'from':receiver_address})
print("Transaction Hash:",_.hex())

Transaction Hash: 0xbb9b628f49ec8d6f24b9b870307d54ca24f54091c71792f8cd55ed4015d36539


In [99]:
print(f'''
N                    : {contract_instance.functions.N().call()}
s                    : {contract_instance.functions.s().call()}
f                    : {contract_instance.functions.f().call()}
d                    : {contract_instance.functions.d().call()}
flagContinue         : {contract_instance.functions.flagContinue().call()}
''')


N                    : 20
s                    : 16
f                    : 4
d                    : 0
flagContinue         : False



In [100]:
#verification
assert(contract_instance.functions.N().call() == contract_instance.functions.s().call() + contract_instance.functions.f().call() + contract_instance.functions.d().call())
print(True)

True
